In [ ]:
import os

# использованием PyTorch как фреймворка
os.environ["KERAS_BACKEND"] = "torch"

import torch
import keras
from keras import layers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pandas_datareader.data as web

# Нормировка данных
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler

# Валидация модели

In [ ]:
# Подготовка датасета для обучения в формате inputs, outputs
def make_datasets(input_data, n_inputs=2, n_outputs=1, gap=0):
	L = len(input_data)
	y = np.full((L-n_inputs-n_outputs-gap, n_outputs), 0.0)
	X = np.full((L-n_inputs-n_outputs-gap, n_inputs), 0.0)

	for i in range(n_inputs):
		X[:,i] = input_data[i:L-n_inputs-n_outputs-gap+i]

	for i in range(n_outputs):
		y[:,i] = input_data[n_inputs+gap+i:L-n_outputs+i]

	return X, y

In [ ]:
rate = web.DataReader(name='WGS10YR', data_source='fred', start='2000-01-01')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Переведем ряда массив Numpy
series = rate.iloc[:,0].values

# scaler = MinMaxScaler()
# series = scaler.fit_predict(series)

## Вариант №1

In [ ]:
# задаём ширину окна и горизонт прогнозирования
n_lags, fh= 20, 5

X, y = make_datasets(series, n_inputs=n_lags, n_outputs=fh)

In [ ]:
# Инициализируем модель
model = keras.Sequential()
# Добавляем входной слой и размер батча
model.add(keras.Input(shape=(1,n_lags), batch_size=32))
# Добавляем слой RNN
model.add(layers.SimpleRNN(units=50, activation='relu', seed=0))
# Добавляем слой LSTM
# model.add(layers.LSTM(units=50, activation='relu', seed=0))
# Добавляем слой GRU
# model.add(layers.GRU(units=50, activation='relu', seed=0))
# Задаём выходной слой
model.add(layers.Dense(fh))
# Собираем модель
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mae'])

model.summary()

In [ ]:
# Можно задать для реплицируемои
# keras.utils.set_random_seed(42)

# Зададим деление выборки на тестовую и обучающую в соотношении 20:80
history = model.fit(
    X, y,
    epochs=10,
    batch_size=None,
    shuffle=False,
    verbose=1,
	validation_split = 0.2
)

In [ ]:
# Метрики по эпохам
history.history

## Вариант №2

In [ ]:
# задаём ширину окна и горизонт прогнозирования
n_lags, fh= 20, 5

X, y = make_datasets(series, n_inputs=n_lags, n_outputs=fh)

In [ ]:
# 
test_size = 30
X_train, y_train = X[:-test_size,:], y[:-test_size,:]
X_test, y_test = X[-test_size:,:], y[-test_size:,:]

In [ ]:
# Инициализируем модель
model = keras.Sequential()
# Добавляем входной слой и размер батча
model.add(keras.Input(shape=(1,n_lags), batch_size=32))
# Добавляем слой RNN
model.add(layers.SimpleRNN(units=50, activation='relu', seed=0))
# Добавляем слой LSTM
# model.add(layers.LSTM(units=50, activation='relu', seed=0))
# Добавляем слой GRU
# model.add(layers.GRU(units=50, activation='relu', seed=0))
# Задаём выходной слой
model.add(layers.Dense(fh))
# Собираем модель
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mae'])

model.summary()

In [ ]:
# Можно задать для реплицируемои
# keras.utils.set_random_seed(42)

# Зададим деление выборки на тестовую и обучающую в соотношении 20:80
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=None,
    shuffle=False,
    verbose=1,
	validation_split = 0.2
)

In [ ]:
model.evaluate(X_test,y_test)